In [136]:
# imports

import json
import requests
import numpy
import pandas as pd
import datetime

In [137]:
# given strings

api_token = 'b396409266b327e2d17cb6e1540c'
post_url = 'https://candidate.hubteam.com/candidateTest/v3/problem/result?userKey=b396409266b327e2d17cb6e1540c'
api_url_base = 'https://candidate.hubteam.com/candidateTest/v3/problem/dataset?userKey=b396409266b327e2d17cb6e1540c'

In [138]:
# from https://www.digitalocean.com/community/tutorials/how-to-use-web-apis-in-python-3
# collect data 

headers = {'Content-Type': 'application/json',
           'Authorization': 'Bearer {0}'.format(api_token)}



def get_partners_info():
    
    response = requests.get(api_url_base, headers=headers)

    if response.status_code != 200:
        print(response.status_code)
        return None
        
    loaded_json = json.loads(response.text)
    
    return loaded_json

In [139]:
# sort json 
string_in_loaded_json = 'partners'
countries = {}

def sort_info(json):
    
    for person in json[string_in_loaded_json]:
        if person['country'] not in countries:
            countries[person['country']] = [person]
        else:
            countries[person['country']].append(person)


In [140]:
# algorithm 

import dateutil.parser as parser 

available_dates_string = 'availableDates'

def peoples_common_dates(adjacent_days, country):
    
    final_dates = [] 
    dates_dict = {} 
    
    for day in adjacent_days:
        
        if day not in dates_dict:
            dates_dict[day] = 0
            
        for person in countries[country]:
            if day.strftime('%Y-%m-%d') and (day+timedelta(days=1)).strftime('%Y-%m-%d') in person[available_dates_string]:
                dates_dict[day] += 1
                
                
    final_dates.append(adjacent_days[0])
    for day in dates_dict.keys():
        if dates_dict[day] > dates_dict[final_dates[0]]:
            final_dates.pop()
            final_dates.append(day)
        if dates_dict[day] == dates_dict[final_dates[0]]:
            if day < final_dates[0]:
                final_dates.pop()
                final_dates.append(day)

    
    if dates_dict[final_dates[0]] == 0:
        final_dates[0] = None
        
            
    final_dates.append((final_dates[0]+timedelta(days=1)).strftime('%Y-%m-%d'))
    final_dates[0] = final_dates[0].strftime('%Y-%m-%d')
    
    return final_dates



def find_adjacent_days(array):
    
    adjacent_days = []
    
    for i in range(len(array) - 1):
        newDate = dateutil.parser.parse(array[i])
        newDatePlusOne = dateutil.parser.parse(array[i+1])
        
        if (newDate + timedelta(days=1)) == newDatePlusOne:
            
            if newDate not in adjacent_days:
                adjacent_days.append(newDate)
                
    return adjacent_days
    

def sorting_algo():
    
    potential_dates_for_each_country = {} 
    final_dates_for_each_country = {}
    
    
    for country in countries.keys():
        
        if country not in potential_dates_for_each_country:
            potential_dates_for_each_country[country] = [] 
            
        array_of_dates = []
        for person in countries[country]:
            
            for date in person[available_dates_string]:
                array_of_dates.append(date)
            
        final_dates_for_each_country[country] = peoples_common_dates(find_adjacent_days(array_of_dates), country)
        
    return final_dates_for_each_country

In [141]:
# posting


dates = sorting_algo()
data = {}
data['countries'] = []
attendees = {}
    
    
for country in dates:

        
    for person in countries[country]:
        if country not in attendees:
            attendees[country] = []
        if dates[country][0] and dates[country][1] in person[available_dates_string]:
            attendees[country].append(person['email'])
            
            
    
for country in countries:
    data['countries'].append({'attendeeCount' : len(attendees[country]), 
                              'attendees' : attendees[country],
                              'name' : country, 
                              'startDate' : dates[country][0] 
                             })


x = requests.post(post_url, data=json.dumps(data))

print(x.text)


{"status":"error","message":"Results don't match.","correlationId":"94916571-ed1e-4b13-ad65-acbf167f86d4","requestId":"34400c5a-04c9-4444-8c28-5b1a62986965"}
